In [1]:
# import necessary packages
import os, sys
import glob
import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from ipywidgets import FloatProgress
from PIL import Image
from scipy import ndimage
from importlib import reload
from DGXutils import GetFileNames

sys.path.append('../')

import utils.Preprocessing as Preprocessing

In [3]:
model_type = 'vanilla'

# options
image_path = f"../data/test/{model_type}_test_predictions/pred_"
mask_path = "../data/test/test_masks_preproc/"
image_save_path = f"../data/test/{model_type}_test_predictions_by_pod/pred_"
mask_save_path = "../data/test/test_masks_by_pod/"

plot = False

# load image data
image_names = glob.glob(mask_path + "*.png")
image_names = [os.path.basename(x) for x in image_names]

In [11]:
# split train masks
reload(Preprocessing)
Preprocessing.split_image(image_names=image_names,
                          image_path=image_path,
                          image_save_path=image_save_path,
                          mask_path=mask_path,
                          mask_save_path=mask_save_path,
                          plot=False)


Processing image 5/6:  490-ref-ISU_073-rep_1-W.png


  0%|          | 0/15 [00:00<?, ?it/s]

Image Splitting Complete!
